# Component 1: MLflow Tracking

You can track almost everything using MLflow:  
Hyperparameters : n_estimators, max depth, epochs, learning rate, kernel size, dropout, batch size ..  
Metrics: AUC, MAE, MSE; F1 score, Accuracy ..  
Artifacts:   
 ❏ models: saved on disk in binary format (pickle, h5, pt, etc.)  
 ❏ outputs (other than models, e.g. images, csv, text, etc.)  
Source: which script / notebook started this experiment.  
Tags and Comments: information about the data/features.  

In [ ]:
with mlflow.start_run(run_name="train") as run:

        mlflow.set_tag("mlflow.runName", "train")

        # create model instance: GBRT (Gradient Boosted Regression Tree)
        model = GradientBoostingRegressor(**run_parameters)

        # Model Training
        model.fit(X_train, y_train)

        # get evaluations scores
        score = rmse_score(y_test, model.predict(X_test))
        score_cv = rmse_cv_score(model, X_train, y_train)

        # generate charts
        # model_feature_importance(model, X_train, model_artifacts_dir)

        # log input features
        mlflow.set_tag("features", str(X_train.columns.values.tolist()))

        # Log tracked parameters
        mlflow.log_params(run_parameters)

        mlflow.log_metrics(
            {
                "RMSE_CV": score_cv.mean(),
                "RMSE": score,
            }
        )

        # log training loss
        for s in model.train_score_:
            mlflow.log_metric("Train Loss", s)

        # get model signature
        signature = infer_signature(
            model_input=X_train,
            model_output=model.predict(X_train)
        )

        # Save model to artifacts
        mlflow.sklearn.log_model(model, "model", signature=signature)


log_params() — It logs a parameter under the current run.   
log_metric() — It logs a metric under the current run.  
log_model() — It logs the model as binary file.  
infer_signature() — it specifies the model input/output.  
set_tags() — It logs input features used to train the model.  

## Component 2: MLflow Project

- code packaging structure that is reusable and repeatable  
- This format is documented in a YAML file known as an MLproject file  
- The MLproject file must consist of the three basic components as listed below  
    Name — A human-readable name given for the project.  
    Environment — A software environment that is used to execute project entry points. It includes all library dependencies required by the project code and it supports Conda environment, Docker container environment, as well as system environment.  
    Entry Points — It includes commands and information about parameters. Each project must contain at least one entry point, which is called at the beginning of project execution  

In [ ]:
3 MLproject file
name: mlflow_example

conda_env: conda.yaml

entry_points:

  load_raw_data:
    command: "python preprocess.py"

  train:
    command: "python train.py"

  main:
    command: "python main.py"

*conda.yaml* is another special file that can be used to declare the conda environment needed to run the steps in this pipeline.  
*preprocess.py* will process the raw data into a more training friendly format, performing feature engineering and saving it so that it can be used by next step.  
*train.py* will build a model and train it on the data produced by the previous task. Once training finishes the model is put in the artifact store for later use, e.g. serving.  
*main.py* is the entry point of the pipeline and will be orchestrating the previous steps.

In [ ]:
# main.py file

import logging
import traceback
import warnings

import mlflow
import click


def _run(entrypoint, parameters={}, source_version=None, use_cache=True):
    """Launching new run for an entrypoint"""

    print(
        "Launching new run for entrypoint=%s and parameters=%s"
        % (entrypoint, parameters)
    )
    submitted_run = mlflow.run(".", entrypoint, parameters=parameters)
    return submitted_run


@click.command()
def workflow():
    """run the workflow"""
    with mlflow.start_run(run_name="data-pipeline"):
        mlflow.set_tag("mlflow.runName", "data-pipeline")
        _run("load_raw_data")
        _run("train", {"learning_rate": 0.1, "max_depth": 5})


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    logging.basicConfig(
        level=logging.INFO,
        filename="logs/train_model.log",
        filemode="a",
        format="%(name)s - %(levelname)s - %(asctime)s - %(message)s",
    )
    logger = logging.getLogger(__name__)

    try:
        workflow()
    except Exception as e:
        print("Exception occured. Check logs.")
        logger.error(f"Failed to run workflow due to error:\n{e}")
        logger.error(traceback.format_exc())

# Component 3: MLflow Models

This is the most important step because we can make profit from the model that we built and we make it accessible to users as a real-time serving through a REST API or batch inference on Apache Spark.

Dockerized MLflow model serving (REST API)

In [ ]:
# a python script used to fetch the best model trained and to run mlflow model serving in a docker container
import os
import subprocess
import sys
import warnings

import mlflow

warnings.filterwarnings('ignore')

PROJECT_DIR = sys.path[0]
os.chdir(PROJECT_DIR)

experiment_name = 'Default'
mlflow.set_experiment(experiment_name)

PORT = 5001  # REST API serving port
CONTAINER_NAME = "mlflow_example_model_serving"

best_run_df = mlflow.search_runs(order_by=['metrics.RMSE_CV ASC'], max_results=1)
if len(best_run_df.index) == 0:
    raise Exception(f"Found no runs for experiment '{experiment_name}'")

best_run = mlflow.get_run(best_run_df.at[0, 'run_id'])
best_model_uri = f"{best_run.info.artifact_uri}/model"
# best_model = mlflow.sklearn.load_model(best_model_uri)

# print best run info
print("Best run info:")
print(f"Run id: {best_run.info.run_id}")
print(f"Run parameters: {best_run.data.params}")
print("Run score: RMSE_CV = {:.4f}".format(best_run.data.metrics['RMSE_CV']))
print(f"Run model URI: {best_model_uri}")

# remove current container if exists
subprocess.run(f"docker rm --force {CONTAINER_NAME}", shell=True, check=False, stdout=subprocess.DEVNULL)

# run mlflow model serving in a docker container
docker_run_cmd = f"""
docker run
--name={CONTAINER_NAME}
--volume={PROJECT_DIR}:{PROJECT_DIR}
--publish {PORT}:{PORT}
--interactive
--rm
mlflow_example
mlflow models serve --model-uri {best_model_uri} --host 0.0.0.0 --port {PORT} --workers 2 --no-conda
""".replace('\n', ' ').strip()
print(f"Running command:\n{docker_run_cmd}")

subprocess.run(docker_run_cmd, shell=True, check=True)


You have just to run this command: python3 mlflow_model_driver.py